In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import math
import repl

In [ ]:
repl = repl.Repl()

In [ ]:
repl.run(['load', '--recentre-to-fit-within-nn-bounds'
         ])

In [ ]:
repl.run('cycle 1234')

In [ ]:
NN_W = 16
NN_H = 16
gfx_pos_bb_left = -1
gfx_pos_bb_right = 1 + (NN_W-1) * 2 + (NN_H-1)
gfx_pos_bb_bottom = -1
gfx_pos_bb_top = 1 + (NN_H-1) * math.sqrt(3)
gfx_pos_centre_x = (gfx_pos_bb_left + gfx_pos_bb_right) / 2
gfx_pos_centre_y = (gfx_pos_bb_bottom + gfx_pos_bb_top) / 2
gfx_width = gfx_pos_bb_right - gfx_pos_bb_left
gfx_height = gfx_pos_bb_top - gfx_pos_bb_bottom
gfx_scale = 25
repl.run(['render',
          '--filename', '/tmp/output.png',
          '--offset', ','.join(map(str, [-gfx_pos_centre_x, -gfx_pos_centre_y])),
          '--scale', str(gfx_scale),
          '--size', 'x'.join([str(int(math.ceil(f))) for f in [gfx_width * gfx_scale, gfx_height * gfx_scale]])])

In [ ]:
repl.run(['nn.save_input', '/tmp/inputs.npz'])

In [ ]:
import numpy as np
inputs = np.load('/tmp/inputs.npz')
inputs

In [ ]:
del repl

In [ ]:
import matplotlib
import matplotlib.pyplot
import matplotlib.collections
%matplotlib inline
from PIL import Image

In [ ]:
img = Image.open('/tmp/output.png')
img = img.transpose(Image.Transpose.FLIP_TOP_BOTTOM)
fig = matplotlib.pyplot.figure()
ax = fig.subplots()
ax.imshow(img, origin='lower', extent=(gfx_pos_bb_left, gfx_pos_bb_right, gfx_pos_bb_bottom, gfx_pos_bb_top))

def hexplot(ax, data, cmap):
    def verts(x, y):
        patch = matplotlib.patches.RegularPolygon((x*2 + y, math.sqrt(3) * y), 6, radius=math.sqrt(4/3), color=cmap(data[x,y], alpha=0.5))
        return patch
    c = matplotlib.collections.PatchCollection([verts(x, y) for x in range(NN_W) for y in range(NN_H)], match_original=True)
    return c

print(inputs['spatiotemporal'].shape)
cmap = matplotlib.colormaps['viridis']
feature_id = 0
ax.add_collection(hexplot(ax, inputs['spatiotemporal'][0, feature_id, 0], cmap))
fig.colorbar(matplotlib.cm.ScalarMappable(cmap=cmap),
             ax=ax,
             orientation='horizontal')
matplotlib.pyplot.show()